In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, LeakyReLU, Activation, BatchNormalization
from keras.utils import np_utils
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split, StratifiedKFold
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

import numpy as np
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [3]:
train_images = train_images.reshape(train_images.shape[0], train_images.shape[1], train_images.shape[2], 1).astype('float32') / 255
test_images = test_images.reshape(test_images.shape[0], test_images.shape[1], test_images.shape[2], 1).astype('float32') / 255
print(train_images.shape)
print(test_images.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)


In [4]:
train_labels = np_utils.to_categorical(train_labels, 10)
test_labels = np_utils.to_categorical(test_labels, 10)
print(train_labels.shape)
print(test_labels.shape)

(60000, 10)
(10000, 10)


In [5]:
# kernel_size  2 by 2       Acc : 91.49   
#              3 by 3       Acc : 92.14
# leaky_ReLU   use        
#               (a = 0.1)   Acc : 92.14  
#               (a = 0.01)  Acc : 91.84
#              not use      Acc : 91.79
# use BN       use          Acc : 
#              not use      Acc : 93.37

# Model 1
model1 = Sequential()
model1.add(Conv2D(filters = 32, kernel_size = (3, 3), padding = 'same', input_shape = (28, 28, 1)))
model1.add(LeakyReLU(alpha=0.1))
model1.add(BatchNormalization(axis=-1))
model1.add(Conv2D(filters = 32, kernel_size = (3, 3), padding = 'same'))
model1.add(LeakyReLU(alpha=0.1))
model1.add(BatchNormalization(axis=-1))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Dropout(0.25))

model1.add(Conv2D(filters = 64, kernel_size = (3, 3), padding = 'same'))
model1.add(LeakyReLU(alpha=0.1))
model1.add(BatchNormalization(axis=-1))
model1.add(Conv2D(filters = 64, kernel_size = (3, 3), padding = 'same'))
model1.add(LeakyReLU(alpha=0.1))
model1.add(BatchNormalization(axis=-1))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Dropout(0.25))

model1.add(Conv2D(filters = 128, kernel_size = (3, 3), padding = 'same'))
model1.add(LeakyReLU(alpha=0.1))
model1.add(BatchNormalization(axis=-1))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Dropout(0.25))

model1.add(Flatten())
model1.add(Dense(512))
model1.add(Activation('relu'))
model1.add(BatchNormalization(axis=-1))
model1.add(Dropout(0.5))

model1.add(Dense(10))
model1.add(Activation('softmax'))

W0809 08:03:58.308495 140393059452800 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0809 08:03:58.352232 140393059452800 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0809 08:03:58.359042 140393059452800 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0809 08:03:58.396958 140393059452800 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0809 08:03:58.397863 1403930594

In [0]:
# Model 2
model2 = Sequential()
model2.add(Conv2D(filters = 32, kernel_size = (2, 2), padding = 'same', input_shape = (28, 28, 1)))
model2.add(LeakyReLU(alpha=0.1))
model2.add(BatchNormalization(axis=-1))
model2.add(Conv2D(filters = 32, kernel_size = (2, 2), padding = 'same'))
model2.add(LeakyReLU(alpha=0.1))
model2.add(BatchNormalization(axis=-1))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Dropout(0.25))

model2.add(Conv2D(filters = 64, kernel_size = (2, 2), padding = 'same'))
model2.add(LeakyReLU(alpha=0.1))
model2.add(BatchNormalization(axis=-1))
model2.add(Conv2D(filters = 64, kernel_size = (2, 2), padding = 'same'))
model2.add(LeakyReLU(alpha=0.1))
model2.add(BatchNormalization(axis=-1))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Dropout(0.25))

model2.add(Conv2D(filters = 128, kernel_size = (2, 2), padding = 'same'))
model2.add(LeakyReLU(alpha=0.1))
model2.add(BatchNormalization(axis=-1))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Dropout(0.25))

model2.add(Flatten())
model2.add(Dense(512))
model2.add(Activation('relu'))
model2.add(BatchNormalization(axis=-1))
model2.add(Dropout(0.5))

model2.add(Dense(10))
model2.add(Activation('softmax'))

In [0]:
# test size   0.1    Acc : 91.72
#             0.2    Acc : 92.19
#             0.3    Acc : 91.38
train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.2, shuffle=True)

In [0]:
models = []
models.append(model1)
models.append(model2)

In [9]:
for n_model in models:
  n_model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
# default :     Acc : 91.88
# lr = 0.01     Acc : 87.99
# lr = 0.001    Acc : 92.17
# lr = 0.0001   Acc : 91.52

W0809 08:04:02.811054 140393059452800 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [0]:
# patience  2    Acc : 92.14
#           5    Acc : 92.17
#           10   Acc : 92.39
callbacks = [EarlyStopping(monitor='val_loss', patience=10), ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

In [11]:
# epoch    20    Acc : 92.14
#          30    Acc : 92.02
#          40    Acc : 92.29
#          50    Acc : 92.37
#          70    Acc : 92.14
for n_model in models:
  n_model.fit(train_images, train_labels, epochs=100, batch_size=128, validation_data=(val_images, val_labels), callbacks = callbacks)  # 마지막엔 100정도

W0809 08:04:02.962446 140393059452800 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 48000 samples, validate on 12000 samples
Epoch 1/100
48000/48000 [==============================] - 13s 265us/step - loss: 0.6227 - acc: 0.7868 - val_loss: 0.3500 - val_acc: 0.8730
Epoch 2/100
48000/48000 [==============================] - 8s 158us/step - loss: 0.3679 - acc: 0.8653 - val_loss: 0.3096 - val_acc: 0.8899
Epoch 3/100
48000/48000 [==============================] - 8s 158us/step - loss: 0.3149 - acc: 0.8841 - val_loss: 0.2706 - val_acc: 0.9018
Epoch 4/100
48000/48000 [==============================] - 8s 158us/step - loss: 0.2800 - acc: 0.8977 - val_loss: 0.2621 - val_acc: 0.9070
Epoch 5/100
48000/48000 [==============================] - 8s 158us/step - loss: 0.2629 - acc: 0.9039 - val_loss: 0.2374 - val_acc: 0.9112
Epoch 6/100
48000/48000 [==============================] - 8s 158us/step - loss: 0.2498 - acc: 0.9081 - val_loss: 0.3043 - val_acc: 0.8922
Epoch 7/100
48000/48000 [==============================] - 8s 158us/step - loss: 0.2351 - acc: 0.9127 - val_loss: 0

In [12]:
for n_model in models:
  print("Accuracy : %.4f" % (n_model.evaluate(test_images, test_labels)[1]))

10000/10000 [==============================] - 1s 82us/step
Accuracy : 0.9345
10000/10000 [==============================] - 1s 82us/step
Accuracy : 0.9348


In [13]:
labels = []
for n_model in models:
    predicts = np.argmax(n_model.predict(test_images), axis=1)
    labels.append(predicts)   # 각 model에서의 predicted label을 label에 넣습니다.
print(labels)

[array([9, 2, 1, ..., 8, 1, 5]), array([9, 2, 1, ..., 8, 1, 5])]


In [14]:
import scipy
from scipy import stats

labels = np.array(labels)   
print(labels)
labels = np.transpose(labels, (1, 0))  # 각 column은 모델들이 특정 이미지에 대해 voting한 값들의 모임이 됩니다.
print(labels)
labels = scipy.stats.mode(labels, axis=1)[0]   # scipy.stats.mode => 세 모델이 내놓은 prediction 중 가장 보편적인 값을 찾습니다.
print(labels)
labels = np.squeeze(labels)   # 쓸모없는 차원을 제거합니다.
print(labels)

[[9 2 1 ... 8 1 5]
 [9 2 1 ... 8 1 5]]
[[9 9]
 [2 2]
 [1 1]
 ...
 [8 8]
 [1 1]
 [5 5]]
[[9]
 [2]
 [1]
 ...
 [8]
 [1]
 [5]]
[9 2 1 ... 8 1 5]


In [15]:
print(len(labels))
print(len(test_labels))
print(test_labels)
test_label = np.argmax(test_labels, axis=1)
print(test_label)

10000
10000
[[0. 0. 0. ... 0. 0. 1.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[9 2 1 ... 8 1 5]


In [16]:
cnt = 0
for i in range(len(labels)):
  if labels[i] == test_label[i]:
    cnt += 1
acc = cnt / len(labels)
print("Accuracy : %.4f" % acc)

Accuracy : 0.9359


위의 코드에서 보시다시피, 변경하였을 때 유의미한 차이를 낼 수 있다고 판단한 파라미터들을 조금씩 변경해가며 최적의 accuracy가 나올 수 있도록 하였습니다.